In [1]:
# Fix PATH to include brew location
import os
os.environ["PATH"] = "/usr/local/bin:" + os.environ.get("PATH", "")
!which brew

/usr/local/bin/brew


In [2]:
!brew install postgresql@15

To reinstall 15.14, run:
  brew reinstall postgresql@15


In [3]:
!brew services start postgresql@15

Service `postgresql@15` already started, use `brew services restart postgresql@15` to restart.


In [4]:
!psql --version

psql (PostgreSQL) 14.19 (Homebrew)


In [5]:
!ps aux | grep postgres | head -5

Ali                645   0.0  0.0 34386920   1440   ??  Ss    7:54AM   0:00.00 postgres: logical replication launcher   
Ali                644   0.0  0.0 34386920   1692   ??  Ss    7:54AM   0:00.05 postgres: autovacuum launcher   
Ali                643   0.0  0.0 34385704    908   ??  Ss    7:54AM   0:00.03 postgres: walwriter   
Ali                641   0.0  0.0 34385704   1552   ??  Ss    7:54AM   0:00.04 postgres: background writer   
Ali                640   0.0  0.0 34385704   2496   ??  Ss    7:54AM   0:00.01 postgres: checkpointer   


In [6]:
!psql postgres -c "SELECT version();"

                                                            version                                                             
--------------------------------------------------------------------------------------------------------------------------------
 PostgreSQL 15.14 (Homebrew) on x86_64-apple-darwin21.6.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
(1 row)



In [7]:
!createdb mydatabase

createdb: error: database creation failed: ERROR:  database "mydatabase" already exists


In [8]:
!psql mydatabase -c "CREATE USER user12 WITH PASSWORD 'pass12';"

ERROR:  role "user12" already exists


In [9]:
!psql mydatabase -c "GRANT ALL ON SCHEMA public TO user12;"

GRANT


In [10]:
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT current_user, current_database();"

 current_user | current_database 
--------------+------------------
 user12       | mydatabase
(1 row)



In [11]:
# List all databases on our PostgreSQL server
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "\l"

                           List of databases
     Name     | Owner  | Encoding | Collate | Ctype | Access privileges 
--------------+--------+----------+---------+-------+-------------------
 mydatabase   | Ali    | UTF8     | en_US   | en_US | =Tc/Ali          +
              |        |          |         |       | Ali=CTc/Ali      +
              |        |          |         |       | user12=CTc/Ali
 postgres     | Ali    | UTF8     | en_US   | en_US | 
 template0    | Ali    | UTF8     | en_US   | en_US | =c/Ali           +
              |        |          |         |       | Ali=CTc/Ali
 template1    | Ali    | UTF8     | en_US   | en_US | =c/Ali           +
              |        |          |         |       | Ali=CTc/Ali
 your_db_name | user12 | UTF8     | en_US   | en_US | 
(5 rows)



In [12]:
# List all tables (should be empty for now)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "\dt"

             List of relations
 Schema |      Name       | Type  | Owner  
--------+-----------------+-------+--------
 public | alembic_version | table | user12
 public | pdfs            | table | user12
 public | todos           | table | user12
(3 rows)



In [13]:
# Create table using string concatenation method
sql_command = "CREATE TABLE test_todos (" + \
              "id SERIAL PRIMARY KEY, " + \
              "name TEXT NOT NULL, " + \
              "completed BOOLEAN DEFAULT FALSE);"

print("Creating table...")
print(sql_command)
print("\nExecuting...")
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

Creating table...
CREATE TABLE test_todos (id SERIAL PRIMARY KEY, name TEXT NOT NULL, completed BOOLEAN DEFAULT FALSE);

Executing...
CREATE TABLE


In [14]:
# Add some test data using triple quotes
sql_command = """
INSERT INTO test_todos (name, completed) VALUES 
('Buy groceries', false),
('Walk the dog', true),
('Learn PostgreSQL', false);
"""

print("SQL Command:")
print(sql_command)
print("\nExecuting...")
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

SQL Command:

INSERT INTO test_todos (name, completed) VALUES 
('Buy groceries', false),
('Walk the dog', true),
('Learn PostgreSQL', false);


Executing...
INSERT 0 3


In [15]:
sql_command = "INSERT INTO test_todos (name, completed) VALUES " + \
              "('Buy groceries', false), " + \
              "('Walk the dog', true), " + \
              "('Learn PostgreSQL', false);"

print("SQL Command:")
print(sql_command)
print("\nExecuting...")
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"


SQL Command:
INSERT INTO test_todos (name, completed) VALUES ('Buy groceries', false), ('Walk the dog', true), ('Learn PostgreSQL', false);

Executing...
INSERT 0 3


In [16]:
# Show all data in our test table
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM test_todos;"


 id |       name       | completed 
----+------------------+-----------
  1 | Buy groceries    | f
  2 | Walk the dog     | t
  3 | Learn PostgreSQL | f
  4 | Buy groceries    | f
  5 | Walk the dog     | t
  6 | Learn PostgreSQL | f
(6 rows)



In [17]:
# Show only incomplete todos
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM test_todos WHERE completed = false;"

 id |       name       | completed 
----+------------------+-----------
  1 | Buy groceries    | f
  3 | Learn PostgreSQL | f
  4 | Buy groceries    | f
  6 | Learn PostgreSQL | f
(4 rows)



In [18]:
# Mark "Learn PostgreSQL" as completed
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "UPDATE test_todos SET completed = true WHERE name = 'Learn PostgreSQL';"

UPDATE 2


In [19]:
# Check that it was updated
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM test_todos;"

 id |       name       | completed 
----+------------------+-----------
  1 | Buy groceries    | f
  2 | Walk the dog     | t
  4 | Buy groceries    | f
  5 | Walk the dog     | t
  3 | Learn PostgreSQL | t
  6 | Learn PostgreSQL | t
(6 rows)



In [20]:
# Delete a specific todo
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "DELETE FROM test_todos WHERE name = 'Walk the dog';"

DELETE 2


In [21]:
# Check what's left
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM test_todos;"

 id |       name       | completed 
----+------------------+-----------
  1 | Buy groceries    | f
  4 | Buy groceries    | f
  3 | Learn PostgreSQL | t
  6 | Learn PostgreSQL | t
(4 rows)



In [22]:
# Remove the test table
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "DROP TABLE test_todos;"

DROP TABLE


In [23]:
# Check that our database exists
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT current_database(), current_user;"

 current_database | current_user 
------------------+--------------
 mydatabase       | user12
(1 row)



In [24]:
# This is what should be in our FastAPI backend .env file
print("""
Required environment variables for our todo app:

DATABASE_HOST=localhost
DATABASE_NAME=mydatabase
DATABASE_USER=user12
DATABASE_PASSWORD=pass12
DATABASE_PORT=5432
APP_NAME="Full Stack To Do App"
""")


Required environment variables for our todo app:

DATABASE_HOST=localhost
DATABASE_NAME=mydatabase
DATABASE_USER=user12
DATABASE_PASSWORD=pass12
DATABASE_PORT=5432
APP_NAME="Full Stack To Do App"



In [25]:
# Test connection with the exact same parameters our app will use
!PGPASSWORD=pass12 psql -U user12 -h localhost -p 5432 -d mydatabase -c "SELECT 'Connection successful!' as status, version();"

         status         |                                                            version                                                             
------------------------+--------------------------------------------------------------------------------------------------------------------------------
 Connection successful! | PostgreSQL 15.14 (Homebrew) on x86_64-apple-darwin21.6.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
(1 row)



In [26]:
# Test that our user can create tables
sql_command = """
CREATE TABLE permission_test (
    id SERIAL PRIMARY KEY,
    test_message TEXT
);
"""

!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

CREATE TABLE


In [27]:
# Test that our user can insert data
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "INSERT INTO permission_test (test_message) VALUES ('Permissions working!');"

INSERT 0 1


In [28]:
# Test that our user can read data
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM permission_test;"

 id |     test_message     
----+----------------------
  1 | Permissions working!
(1 row)



In [29]:
# Clean up our test table
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "DROP TABLE permission_test;"

DROP TABLE


In [30]:
# Show our database and user setup
sql_command = """
SELECT 
'Database: ' || current_database() as info
UNION ALL
SELECT 
'User: ' || current_user
UNION ALL
SELECT 
'Connection: Successful'
UNION ALL
SELECT 
'Ready for todo app: YES!';
"""

print("Database setup summary:")
print(sql_command)
print("\nExecuting...")
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

Database setup summary:

SELECT 
'Database: ' || current_database() as info
UNION ALL
SELECT 
'User: ' || current_user
UNION ALL
SELECT 
'Connection: Successful'
UNION ALL
SELECT 
'Ready for todo app: YES!';


Executing...
           info           
--------------------------
 Database: mydatabase
 User: user12
 Connection: Successful
 Ready for todo app: YES!
(4 rows)



In [31]:
# Create the exact todos table structure our app expects
sql_command="""
CREATE TABLE todos (
    id BIGSERIAL PRIMARY KEY,
    name TEXT,
    completed BOOLEAN NOT NULL DEFAULT FALSE
);
"""


!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

ERROR:  relation "todos" already exists


In [32]:
sql_command = """
INSERT INTO todos (name, completed) VALUES 
('Buy groceries', false),
('Walk the dog', false),
('Finish PostgreSQL setup', true),
('Learn FastAPI', false);
"""

!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"


INSERT 0 4


In [33]:
# SELECT: Read all todos (READ operation)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM todos ORDER BY id;"

 id |          name           | completed 
----+-------------------------+-----------
  3 | Buy groceries           | f
  4 | Walk the dog            | f
  5 | Finish PostgreSQL setup | t
  6 | Learn FastAPI           | f
(4 rows)



In [34]:
# SELECT with filter: Get only incomplete todos
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM todos WHERE completed = false;"

 id |     name      | completed 
----+---------------+-----------
  3 | Buy groceries | f
  4 | Walk the dog  | f
  6 | Learn FastAPI | f
(3 rows)



In [35]:
# UPDATE: Mark a todo as completed (UPDATE operation)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "UPDATE todos SET completed = true WHERE name = 'Buy groceries';"

UPDATE 1


In [36]:
# Verify the update worked
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM todos WHERE name = 'Buy groceries';"

 id |     name      | completed 
----+---------------+-----------
  3 | Buy groceries | t
(1 row)



In [37]:
# DELETE: Remove a todo (DELETE operation)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "DELETE FROM todos WHERE name = 'Walk the dog';"

DELETE 1


In [38]:
# Verify the deletion worked
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM todos ORDER BY id;"

 id |          name           | completed 
----+-------------------------+-----------
  3 | Buy groceries           | t
  5 | Finish PostgreSQL setup | t
  6 | Learn FastAPI           | f
(3 rows)



In [39]:
# Count total todos and completed todos
sql_command = """
SELECT 
    COUNT(*) as total_todos,
    COUNT(CASE WHEN completed = true THEN 1 END) as completed_todos,
    COUNT(CASE WHEN completed = false THEN 1 END) as pending_todos
FROM todos;
"""

!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

 total_todos | completed_todos | pending_todos 
-------------+-----------------+---------------
           3 |               2 |             1
(1 row)



In [40]:
# Get the newest todo (highest ID)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT * FROM todos ORDER BY id DESC LIMIT 1;"

 id |     name      | completed 
----+---------------+-----------
  6 | Learn FastAPI | f
(1 row)



In [41]:
# Show the structure of our todos table
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "\d todos"

                              Table "public.todos"
  Column   |  Type   | Collation | Nullable |              Default              
-----------+---------+-----------+----------+-----------------------------------
 id        | bigint  |           | not null | nextval('todos_id_seq'::regclass)
 name      | text    |           |          | 
 completed | boolean |           | not null | false
Indexes:
    "todos_pkey" PRIMARY KEY, btree (id)



In [42]:
# Insert multiple todos to test performance
sql_command = """
INSERT INTO todos (name, completed) 
SELECT 
    'Test todo ' || generate_series,
    (random() > 0.5)
FROM generate_series(1, 100);
"""

!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "$sql_command"

INSERT 0 100


In [43]:
# Count all todos (should be around 103 now)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT COUNT(*) as total_todos FROM todos;"

 total_todos 
-------------
         103
(1 row)



In [44]:
# Test a fast query with conditions
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT COUNT(*) FROM todos WHERE completed = true;"

 count 
-------
    50
(1 row)



In [45]:
# Remove all test data (start fresh for our app)
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "DELETE FROM todos;"

DELETE 103


In [46]:
# Reset the ID sequence to start from 1 again
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "ALTER SEQUENCE todos_id_seq RESTART WITH 1;"

ALTER SEQUENCE


In [47]:
# Verify the table is empty and ready
!PGPASSWORD=pass12 psql -U user12 -h localhost -d mydatabase -c "SELECT COUNT(*) as remaining_todos FROM todos;"

 remaining_todos 
-----------------
               0
(1 row)

